In [85]:
library(jsonlite)
library(tidyverse)

setwd('C:/Users/Deanna/Documents/Job Stuff/Data Analyst')

https://db.rstudio.com/getting-started/database-queries/#:~:text=To%20use%20SQL%2C%20open%20an,in%20the%20code%20chunk%20options.

# Thought Process

Hey there!

This notebook is a documentation of my thought process as I went through this exercise. Feel free to follow along while I explain how I got from point A to point B (or C).

After reading through the exercise, here's my first brain dump:
- JSON files need to be converted to CSV
- The ER diagram will probably be straightforward with the join between reciepts and users (via userID)
- I need to mess with the data to figure out how to join brands to the reciepts table since the column names and descriptions aren't making the join obvious
- The questions all come in pairs and two can be answered at once in SQL with the use of a temp table, though I would prefer using R for the visualizations but I can do it in SQL. (I really wanted to use ggplot2 ಥ_ಥ )
- for quality issues, we're probably talking looking for outliers, expected values, etc etc. Lots of boxplots for this part.
- from there the write-up should be pretty ezpz!

## Part 1: Get Familiar with the Data and make an ER Diagram

To start, I need to see the data! JSON files are gross imo, but I can turn them into data frames in R and fiddle with them a bit.

In [122]:
brands <- stream_in(file("brands.json"))
receipts <- stream_in(file("receipts.json"))
users <- stream_in(file("users.json"))

opening fileconnectionoldClass input connection.



 Imported 1167 records. Simplifying...


closing fileconnectionoldClass input connection.

opening fileconnectionoldClass input connection.



 Imported 1119 records. Simplifying...


closing fileconnectionoldClass input connection.

opening fileconnectionoldClass input connection.



 Imported 495 records. Simplifying...


closing fileconnectionoldClass input connection.



In [116]:
brands$'_id' <- flatten(brands$'_id')
brands$cpg$`$id` <- flatten(brands$cpg$`$id`)

brands <- brands %>%
            as_tibble() %>%
            unnest(cpg)

brands$'_id' <- as.character(brands$'_id')
brands$`$id` <- as.character(brands$`$id`)

head(brands)

_id,barcode,category,categoryCode,$id,$ref,name,topBrand,brandCode
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<lgl>,<chr>
601ac115be37ce2ead437551,511111019862,Baking,BAKING,601ac114be37ce2ead437550,Cogs,test brand @1612366101024,FALSE,NA
601c5460be37ce2ead43755f,511111519928,Beverages,BEVERAGES,5332f5fbe4b03c9a25efd0ba,Cogs,Starbucks,FALSE,STARBUCKS
601ac142be37ce2ead43755d,511111819905,Baking,BAKING,601ac142be37ce2ead437559,Cogs,test brand @1612366146176,FALSE,TEST BRANDCODE @1612366146176
601ac142be37ce2ead43755a,511111519874,Baking,BAKING,601ac142be37ce2ead437559,Cogs,test brand @1612366146051,FALSE,TEST BRANDCODE @1612366146051
601ac142be37ce2ead43755e,511111319917,Candy & Sweets,CANDY_AND_SWEETS,5332fa12e4b03c9a25efd1e7,Cogs,test brand @1612366146827,FALSE,TEST BRANDCODE @1612366146827
601ac142be37ce2ead43755b,511111719885,Baking,BAKING,601ac142be37ce2ead437559,Cogs,test brand @1612366146091,FALSE,TEST BRANDCODE @1612366146091


In [123]:
receipts$'_id' <- flatten(receipts$'_id')
receipts$'_id' <- as.character(receipts$'_id')

receipts <- receipts %>%
            as_tibble() %>%
            unnest(rewardsReceiptItemList,names_repair = "unique")

head(receipts)

New names:
• `pointsEarned` -> `pointsEarned...9`
• `pointsEarned` -> `pointsEarned...38`


_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned...9,purchaseDate,⋯,targetPrice,competitiveProduct,originalFinalPrice,originalMetaBriteItemPrice,deleted,priceAfterCoupon,metabriteCampaignId,rewardsReceiptStatus,totalSpent,userId
<chr>,<int>,<chr>,<dttm>,<dttm>,<dttm>,<dttm>,<dttm>,<chr>,<dttm>,⋯,<chr>,<lgl>,<chr>,<chr>,<lgl>,<chr>,<chr>,<chr>,<chr>,<chr>
5ff1e1eb0a720f0523000575,500,"Receipt number 2 completed, bonus point schedule DEFAULT (5cefdcacf3693e0b50e83a36)",2021-01-03 09:25:31,2021-01-03 09:25:31,2021-01-03 09:25:31,2021-01-03 09:25:36,2021-01-03 09:25:31,500.0,2021-01-02 18:00:00,⋯,NA,NA,NA,NA,NA,NA,NA,FINISHED,26.00,5ff1e1eacfcf6c399c274ae6
5ff1e1bb0a720f052300056b,150,"Receipt number 5 completed, bonus point schedule DEFAULT (5cefdcacf3693e0b50e83a36)",2021-01-03 09:24:43,2021-01-03 09:24:43,2021-01-03 09:24:43,2021-01-03 09:24:48,2021-01-03 09:24:43,150.0,2021-01-02 09:24:43,⋯,NA,NA,NA,NA,NA,NA,NA,FINISHED,11.00,5ff1e194b6a9d73a3a9f1052
5ff1e1bb0a720f052300056b,150,"Receipt number 5 completed, bonus point schedule DEFAULT (5cefdcacf3693e0b50e83a36)",2021-01-03 09:24:43,2021-01-03 09:24:43,2021-01-03 09:24:43,2021-01-03 09:24:48,2021-01-03 09:24:43,150.0,2021-01-02 09:24:43,⋯,NA,NA,NA,NA,NA,NA,NA,FINISHED,11.00,5ff1e194b6a9d73a3a9f1052
5ff1e1f10a720f052300057a,5,All-receipts receipt bonus,2021-01-03 09:25:37,2021-01-03 09:25:37,NA,2021-01-03 09:25:42,NA,5,2021-01-02 18:00:00,⋯,NA,NA,NA,NA,NA,NA,NA,REJECTED,10.00,5ff1e1f1cfcf6c399c274b0b
5ff1e1ee0a7214ada100056f,5,All-receipts receipt bonus,2021-01-03 09:25:34,2021-01-03 09:25:34,2021-01-03 09:25:34,2021-01-03 09:25:39,2021-01-03 09:25:34,5.0,2021-01-02 18:00:00,⋯,NA,NA,NA,NA,NA,NA,NA,FINISHED,28.00,5ff1e1eacfcf6c399c274ae6
5ff1e1d20a7214ada1000561,5,All-receipts receipt bonus,2021-01-03 09:25:06,2021-01-03 09:25:06,2021-01-03 09:25:11,2021-01-03 09:25:11,2021-01-03 09:25:06,5.0,2021-01-02 09:25:06,⋯,NA,NA,NA,NA,NA,NA,NA,FINISHED,1.00,5ff1e194b6a9d73a3a9f1052


In [101]:
users$'_id' <- flatten(users$'_id')

users <- tibble(users)
head(users)

_id,active,createdDate,lastLogin,role,signUpSource,state
<list>,<lgl>,<dttm>,<dttm>,<chr>,<chr>,<chr>
5ff1e194b6a9d73a3a9f1052,TRUE,2021-01-03 09:24:04,2021-01-03 09:25:37,consumer,Email,WI
5ff1e194b6a9d73a3a9f1052,TRUE,2021-01-03 09:24:04,2021-01-03 09:25:37,consumer,Email,WI
5ff1e194b6a9d73a3a9f1052,TRUE,2021-01-03 09:24:04,2021-01-03 09:25:37,consumer,Email,WI
5ff1e1eacfcf6c399c274ae6,TRUE,2021-01-03 09:25:30,2021-01-03 09:25:30,consumer,Email,WI
5ff1e194b6a9d73a3a9f1052,TRUE,2021-01-03 09:24:04,2021-01-03 09:25:37,consumer,Email,WI
5ff1e194b6a9d73a3a9f1052,TRUE,2021-01-03 09:24:04,2021-01-03 09:25:37,consumer,Email,WI


In [124]:
#write.csv(brands, "brands.csv", row.names=TRUE)
#write.csv(receipts, "receipts.csv", row.names=TRUE)
#write.csv(users, "users.csv", row.names=TRUE)

Okay I definitely felt my soul leave my body while I figured out how to parse the JSON files. But the rest *should* be easy...